In [52]:
import numpy as np
import meshio
from sklearn.neighbors import KDTree
from pylab import *
rcParams['figure.figsize'] = [16,9]
from matplotlib.colors import ListedColormap
mesh = meshio.read("../client/DUCK_YOU.stl")
import pyvista as pv
pv.start_xvfb()

/home/ted/geometric-tools/ModelInspector/notebooks


In [51]:
class ClickTracker:

    def __init__(self,mesh,plotter) -> None:
        self.tris = np.array([ array for c in mesh.cells if c.type in ('triangle',) for array in c.data])
        cells=np.array([ ([3] + c.tolist()) for c in self.tris]).flatten()
        cell_nodes = np.unique(cells.flatten())
        self.cell_points = mesh.points[cell_nodes]

        centers = np.array([ mesh.points[tri].mean(axis=0) for tri in self.tris])
        celltypes = [pv.CellType.TRIANGLE] * len(self.tris)

        self.grid = pv.UnstructuredGrid(cells, celltypes, self.cell_points)

        self.tree = KDTree(centers)
        self.plotter = plotter

        self.prev_cyl = None
        self.points = centers
        source = np.zeros(cell_nodes.shape[0])
        color = np.linspace(0,1,len(source))

        color_map = np.stack([
            color[::-1],
            np.zeros_like(source),
            color
        ]).T
        color_map = ListedColormap(color_map)
        self.colorizer = color_map
        self.actor_grid = self.plotter.add_mesh(self.grid, show_edges=False)
        
        # self.plotter.add_scalar_bar(title="ShotScore")

    
    def __call__(self,point):

        distances,indices = self.tree.query([point])
        tri = self.cell_points[self.tris[indices.item()]]
        dist = point#tri.mean(axis=0)

        a = tri[1]-tri[0]
        b = tri[2]-tri[1]
        normal = np.cross(a,b)
        normal /= np.linalg.norm(normal)
        if self.prev_cyl is not None:
            self.plotter.remove_actor(self.prev_cyl)
            self.plotter.remove_actor(self.actor_grid)
        

        
        cyl = pv.Cylinder(center=dist+normal, direction=normal,
                                    radius=.25, height=2,)
        
        score = ((self.points-np.array(point))**2).sum(1)
        score = (score-score.min())/(score.max()-score.min())
        # self.plotter.update_scalars(score)

        self.prev_cyl = self.plotter.add_mesh(cyl,self.colorizer(1))
        self.actor_grid = self.plotter.add_mesh(self.grid,show_edges=False,scalars=score)
        self.plotter.update()

In [43]:

pl = pv.Plotter(notebook=True)

pl.track_click_position(
    ClickTracker(
        mesh,
        pl,
    ),
    double=True,
    side='left',
)

pl.show(window_size=[1024,1024]) 

Widget(value="<iframe src='http://localhost:42247/index.html?ui=P_0x7f21e628b7c0_17&reconnect=auto' style='wid…